<a href="https://colab.research.google.com/github/benjamin-ravenscroft/CISC474-Project/blob/main/Trading_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Import packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

# first install yahoo finance
!pip install yfinance
!pip install yahoofinancials
import yfinance as yf
from yahoofinancials import YahooFinancials
!pip install stockstats # used to calculate technical indicators
import stockstats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 

#2. Download Financial Data

In [ ]:
data = yf.download(
  tickers = "AXP AMGN AAPL BA CAT CSCO CVX GS HD HON IBM INTC JNJ KO JPM MCD MMM MRK MSFT NKE PG TRV UNH CRM VZ V WBA WMT DIS DOW", # Dow Jones 30 Tickers
  start="2008-01-01",
  end = "2022-11-30",
  interval="1d",
  ignore_tz=True,
  group_by='ticker',
  auto_adjust= True,
  repair=True,
  prepost=False,
  threads=True,
  proxy=None
)

[*********************100%***********************]  30 of 30 completed


In [ ]:
data.head()

AAPL                                                   BA  \
                Open      High       Low     Close      Volume       Open   
Date                                                                        
2008-01-02  6.066476  6.096615  5.861896  5.931611  1079178800  64.177873   
2008-01-03  5.948964  6.009242  5.866157  5.934350   842066400  63.811413   
2008-01-04  5.828406  5.875594  5.446036  5.481350  1455832000  63.261781   
2008-01-07  5.517884  5.589426  5.182396  5.407983  2072193200  63.210456   
2008-01-08  5.484092  5.554722  5.199750  5.213450  1523816000  60.909255   

                                                      ...        NKE  \
                 High        Low      Close   Volume  ...       Open   
Date                                                  ...              
2008-01-02  64.375747  63.027259  63.481644  4303000  ...  13.339365   
2008-01-03  64.229151  63.569563  63.745457  3427900  ...  13.123677   
2008-01-04  63.877399  62.800072  62.895344  5393400  ...  12.907994   
2008-01-07  63.247102  60.220325  60.733341  9961800  ...  12.858216   
2008-01-08  61.099804  58.373507  58.564056  9810600  ...  12.941172   

                                                             UNH             \
                 High        Low      Close    Volume       Open       High   
Date                                                                          
2008-01-02  13.418173  13.003392  13.123678   9781600  47.164085  47.716629   
2008-01-03  13.221151  12.937026  13.005465  10636000  46.801224  47.246558   
2008-01-04  13.040724  12.796003  12.804298  22335200  46.644541  46.949676   
2008-01-07  12.924581  12.713042  12.881029  20808400  45.803346  47.098111   
2008-01-08  13.061459  12.762816  12.822960  16714400  47.048633  47.403250   

                                           
                  Low      Close   Volume  
Date                                       
2008-01-02  46.562060  46.735245  7698700  
2008-01-03  46.611546  46.949669  4864000  
2008-01-04  46.133230  46.182713  7250600  
2008-01-07  45.803346  46.908432  6093200  
2008-01-08  46.166214  46.355892  8142700  

[5 rows x 150 columns]

In [ ]:
data.shape

(3755, 150)

There is OHLCV (Open, High, Low, Close, Volume) data for 30 stocks = 150 columns, and 3502 days (rows)

# 3. Preprocess Data

Important steps:

1. Check for missing data

2. Add chosen technical indicators to use as features

In [ ]:
data_copy = data.copy()

In [ ]:
ticker_list = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']

In [ ]:
def add_indicators_reshape_df(ticker_list, indicator_list):
  stock_df_list = []
  for i in ticker_list:
    temp = data_copy[i]
    temp['tick'] = i
    stock_df_list.append(temp)

  new_df = pd.concat(stock_df_list, sort=False).sort_index()
  new_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Open,High,Low,Close,Volume,tick
Date,,,,,,
2008-01-02,40.999953,41.180985,39.976725,40.173500,8053700.0,AXP
2008-01-02,34.043740,34.201716,33.361572,33.677525,19918600.0,WMT
2008-01-02,64.473485,64.473485,61.622198,61.929806,9940661.0,IBM
2008-01-02,27.419730,27.682730,26.885249,27.012506,9269900.0,DIS
2008-01-02,47.164085,47.716629,46.562060,46.735245,7698700.0,UNH
...,...,...,...,...,...,...
2022-11-29,38.020000,38.369999,37.950001,38.340000,14007600.0,VZ
2022-11-29,209.350006,210.149994,207.350006,209.059998,8986500.0,V
2022-11-29,40.900002,41.209999,40.630001,41.080002,3212500.0,WBA


Test

In [ ]:
ex = data['BA']
ex['tick'] = 'BA'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
from stockstats import wrap

df = wrap(ex)
df[['macd', 'rsi']]

,macd,rsi
Date,,
2008-01-02,0.000000,NaN
2008-01-03,0.005919,100.000000
2008-01-04,-0.018820,22.369941
2008-01-07,-0.107669,7.155745
2008-01-08,-0.240002,4.124582
...,...,...
2022-11-22,8.609049,66.803305
2022-11-23,8.398676,68.843452
2022-11-25,8.418143,71.646294


In [ ]:
d=pd.DataFrame(df['macd'])

In [ ]:
a = ex.merge(df['macd'], how="left")
a.set_index(ex.index, inplace=True)

In [ ]:
a

,open,high,low,close,volume,tick,macd,macds,macdh,rs_14,rsi
Date,,,,,,,,,,,
2008-01-02,64.177873,64.375747,63.027259,63.481644,4303000,BA,0.000000,0.000000,0.000000,NaN,NaN
2008-01-03,63.811413,64.229151,63.569563,63.745457,3427900,BA,0.005919,0.003288,0.002631,inf,100.000000
2008-01-04,63.261781,63.877399,62.800072,62.895344,5393400,BA,-0.018820,-0.005773,-0.013048,0.288161,22.369941
2008-01-07,63.210456,63.247102,60.220325,60.733341,9961800,BA,-0.107669,-0.040290,-0.067379,0.077073,7.155745
2008-01-08,60.909255,61.099804,58.373507,58.564056,9810600,BA,-0.240002,-0.099700,-0.140302,0.043020,4.124582
...,...,...,...,...,...,...,...,...,...,...,...
2022-11-22,173.649994,174.990005,171.000000,172.500000,4370600,BA,8.609049,8.378909,0.230140,2.012348,66.803305
2022-11-23,172.089996,175.000000,171.770004,174.850006,3523500,BA,8.398676,8.382862,0.015814,2.209598,68.843452
2022-11-25,174.350006,178.460007,174.029999,178.360001,3476900,BA,8.418143,8.389918,0.028224,2.526876,71.646294


In [ ]:
data_c = data.copy()

In [ ]:
a = data_c['AAPL']
a['tick'] = 'AAPL'
b = data_c['AMGN']
b['tick'] = 'AMGN'

new = pd.concat([a,b], sort=False).sort_index()

new.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Open,High,Low,Close,Volume,tick
Date,,,,,,
2008-01-02,6.066476,6.096615,5.861896,5.931611,1079178800,AAPL
2008-01-02,34.931568,35.261396,34.676703,34.931568,7934400,AMGN
2008-01-03,5.948964,6.009242,5.866157,5.934350,842066400,AAPL
2008-01-03,34.804148,35.306382,33.919614,34.249439,11976300,AMGN
2008-01-04,5.828406,5.875594,5.446036,5.481350,1455832000,AAPL


In [ ]:
l = []
for i in ticker_list:
  d = data_c[i]
  d['tick'] = i
  l.append(d)

l 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


[                  Open        High         Low       Close    Volume tick
 Date                                                                     
 2008-01-02   40.999953   41.180985   39.976725   40.173500   8053700  AXP
 2008-01-03   40.299432   40.535561   39.512334   39.677624   7753200  AXP
 2008-01-04   39.347060   39.347060   38.339572   38.678024   8501400  AXP
 2008-01-07   38.984983   39.433628   38.245111   38.851177  14031100  AXP
 2008-01-08   39.118793   39.118793   37.591821   37.741371  11721000  AXP
 ...                ...         ...         ...         ...       ...  ...
 2022-11-22  154.100006  155.149994  153.289993  154.580002   2124600  AXP
 2022-11-23  153.830002  154.990005  152.880005  153.929993   1404400  AXP
 2022-11-25  153.520004  154.470001  153.169998  154.149994    665800  AXP
 2022-11-28  152.690002  153.779999  150.369995  150.869995   1782900  AXP
 2022-11-29  150.960007  154.470001  150.529999  154.419998   2655400  AXP
 
 [3755 rows x 6 columns